In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
from transformers import (
    AutoTokenizer,
    T5ForConditionalGeneration,
    AutoConfig,
)
from pytorch_lightning import LightningModule
import torch

/home/husein/.local/lib/python3.8/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/home/husein/.local/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


2023-08-03 00:35:55.653928: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-03 00:35:55.747042: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-03 00:35:56.472247: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-08-03 00:35:56.472292: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not l

In [3]:
tokenizer = AutoTokenizer.from_pretrained('./out-base-1.1')

In [4]:
tokenizer.push_to_hub('nanot5-base-malaysian-cased', organization='mesolitica')

/home/husein/.local/lib/python3.8/site-packages/transformers/utils/hub.py:665: UserWarning: The `organization` argument is deprecated and will be removed in v5 of Transformers. Set your organization directly in the `repo_id` passed instead (`repo_id={organization}/{model_id}`).
  warnings.warn(


CommitInfo(commit_url='https://huggingface.co/mesolitica/nanot5-base-malaysian-cased/commit/1f58761eeb0f79a2fdf8eacfc8ed31c22d45a115', commit_message='Upload tokenizer', commit_description='', oid='1f58761eeb0f79a2fdf8eacfc8ed31c22d45a115', pr_url=None, pr_revision=None, pr_num=None)

In [5]:
class Module(LightningModule):
    def __init__(self):
        super().__init__()
        config = AutoConfig.from_pretrained(
            './out-base-1.1'
        )
        self.model = T5ForConditionalGeneration.from_pretrained(
            './out-base-1.1',
            config=config,
        )

In [6]:
!ls -lh logs/base-v3

total 8.4G
-rw-r--r-- 1 husein husein 2.8G Ogos  2 12:50 'model-epoch=00-step=290000.ckpt'
-rw-r--r-- 1 husein husein 2.8G Ogos  2 17:01 'model-epoch=00-step=300000.ckpt'
-rw-r--r-- 1 husein husein 2.8G Ogos  2 21:12 'model-epoch=00-step=310000.ckpt'


In [7]:
model = Module()

In [8]:
weights = model.state_dict()

In [9]:
old_weights = torch.load('logs/base-v3/model-epoch=00-step=310000.ckpt',
                             map_location=torch.device('cpu'))['state_dict'].items()

In [10]:
for k, v in old_weights:
    new_k = k.replace('._orig_mod', '')
    print(k, new_k)
    weights[new_k] = v

model._orig_mod.shared.weight model.shared.weight
model._orig_mod.encoder.embed_tokens.weight model.encoder.embed_tokens.weight
model._orig_mod.encoder.block.0.layer.0.SelfAttention.q.weight model.encoder.block.0.layer.0.SelfAttention.q.weight
model._orig_mod.encoder.block.0.layer.0.SelfAttention.k.weight model.encoder.block.0.layer.0.SelfAttention.k.weight
model._orig_mod.encoder.block.0.layer.0.SelfAttention.v.weight model.encoder.block.0.layer.0.SelfAttention.v.weight
model._orig_mod.encoder.block.0.layer.0.SelfAttention.o.weight model.encoder.block.0.layer.0.SelfAttention.o.weight
model._orig_mod.encoder.block.0.layer.0.SelfAttention.relative_attention_bias.weight model.encoder.block.0.layer.0.SelfAttention.relative_attention_bias.weight
model._orig_mod.encoder.block.0.layer.0.layer_norm.weight model.encoder.block.0.layer.0.layer_norm.weight
model._orig_mod.encoder.block.0.layer.1.DenseReluDense.wi_0.weight model.encoder.block.0.layer.1.DenseReluDense.wi_0.weight
model._orig_mod.en

In [11]:
model.load_state_dict(weights)

<All keys matched successfully>

In [12]:
model.model.push_to_hub('nanot5-base-malaysian-cased', organization='mesolitica')

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/mesolitica/nanot5-base-malaysian-cased/commit/fd78a2863cd4dacb40ca5fad9aaa5eb2b32b9d24', commit_message='Upload T5ForConditionalGeneration', commit_description='', oid='fd78a2863cd4dacb40ca5fad9aaa5eb2b32b9d24', pr_url=None, pr_revision=None, pr_num=None)